# **Exploratory Data Analysis of TCGA Data from GDC API**
TCGA: The Cancer Genome Atlas  
GDC: Genomic Data Commons

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd

# Constants and Paths

In [2]:
# Endpoint for files download
DATA_ENDPOINT = 'https://api.gdc.cancer.gov/data'

# Data folder path
DATA_PATH = '../../data'

# External data folder path
EXTERNAL_DATA_PATH = f'{DATA_PATH}/external/gdc-tcga'

# Interim data folder path
INTERIM_DATA_PATH = f'{DATA_PATH}/interim/gdc-tcga'

# Data Loading and Preparation

In [3]:
# DataFrame with TCGA projects
df_projects = pd.read_csv(f'{INTERIM_DATA_PATH}/tcga-projects.csv')

# DataFrame with the cases of interest
df_cases = pd.read_csv(f'{INTERIM_DATA_PATH}/tcga-cases-of-interest.csv')

# DataFrame with the files of interest
df_files = pd.read_csv(f'{INTERIM_DATA_PATH}/tcga-files-of-interest.csv')

In [4]:
# Join both cases and files DataFrames for the exploratory data analysis (EDA)
df_eda = df_cases \
    .merge(
        right=df_files,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Create an acronym column for the file data type
df_eda['data_acronym'] = np.where(
    df_eda['data_type'] == 'Gene Expression Quantification', 'RNA', 'microRNA'
)

# Create an acronym column that combines file data type and sample tissue type
df_eda['data_and_sample'] = (
    df_eda['data_acronym'] + '-' + df_eda['tissue_type']
)

In [5]:
# Print the EDA DataFrame partially
df_eda.head(3)

,case_id,case_primary_site,case_disease_type,project_id,file_id,data_format,data_type,sample_type,tissue_type,created_datetime,updated_datetime,data_acronym,data_and_sample
0,4298ccdb-2e6d-4267-822d-75b021364084,Kidney,Adenomas and Adenocarcinomas,TCGA-KIRC,7ebb3c64-2d8b-4388-8ad8-555fb7cea1b4,TXT,Isoform Expression Quantification,Primary Tumor,Tumor,2018-03-20T00:16:05.303968-05:00,2024-07-29T21:20:32.946706-05:00,microRNA,microRNA-Tumor
1,4298ccdb-2e6d-4267-822d-75b021364084,Kidney,Adenomas and Adenocarcinomas,TCGA-KIRC,6d0df9bf-69d6-447f-b2d9-76391e95d61d,TSV,Gene Expression Quantification,Primary Tumor,Tumor,2021-12-13T19:31:32.504341-06:00,2024-07-30T11:11:25.450940-05:00,RNA,RNA-Tumor
2,a2663a86-a006-4867-9e88-2b523df48303,Kidney,Adenomas and Adenocarcinomas,TCGA-KIRC,9f7e5a27-9a09-474e-a2d6-ed478a124fe3,TXT,Isoform Expression Quantification,Primary Tumor,Tumor,2018-03-19T23:43:21.888100-05:00,2024-07-29T21:23:47.515560-05:00,microRNA,microRNA-Tumor


# Objects Quantification

In [6]:
# Total number of projects of interest
df_eda['project_id'].nunique()

32

In [7]:
# Total number of cases of interest
df_eda['case_id'].nunique()

10207

In [8]:
# Total number of files of interest
df_eda['file_id'].nunique()

21714

# Projects

## Cases and Files per Project

In [9]:
# Count the total number of cases and files per project 
df_eda \
    .groupby('project_id') \
    .agg(
        case_count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
        file_count=pd.NamedAgg(column='file_id', aggfunc='nunique')
    ) \
    .sort_values(by='case_count', ascending=False) \
    .reset_index()

,project_id,case_count,file_count
0,TCGA-BRCA,1097,2424
1,TCGA-UCEC,559,1166
2,TCGA-KIRC,534,1228
3,TCGA-HNSC,528,1131
4,TCGA-OV,522,987
5,TCGA-LUAD,519,1163
6,TCGA-LGG,516,1028
7,TCGA-THCA,507,1129
8,TCGA-LUSC,504,1095
9,TCGA-PRAD,498,1103


## Sample Types and Data File Types per Project

In [10]:
# Count the total number of cases and files per project
pd.set_option('display.max_rows', 500)
df_eda \
    .groupby(['project_id', 'sample_type', 'data_acronym']) \
    .agg(
        case_count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
        file_count=pd.NamedAgg(column='file_id', aggfunc='nunique')
    ) \
    .sort_values(by=['project_id', 'sample_type'], ascending=True)

case_count  file_count
project_id sample_type         data_acronym                        
TCGA-ACC   Primary Tumor       RNA                   79          79
                               microRNA              80          80
TCGA-BLCA  Primary Tumor       RNA                  406         412
                               microRNA             409         417
           Solid Tissue Normal RNA                   19          19
                               microRNA              19          19
TCGA-BRCA  Primary Tumor       RNA                 1095        1111
                               microRNA            1078        1096
           Solid Tissue Normal RNA                  113         113
                               microRNA             104         104
TCGA-CESC  Primary Tumor       RNA                  304         304
                               microRNA             307         307
           Solid Tissue Normal RNA                    3           3
                               microRNA               3           3
TCGA-CHOL  Primary Tumor       RNA                   35          35
                               microRNA              36          36
           Solid Tissue Normal RNA                    9           9
                               microRNA               9           9
TCGA-COAD  Primary Tumor       RNA                  458         481
                               microRNA             444         455
           Solid Tissue Normal RNA                   41          41
                               microRNA               8           8
TCGA-DLBC  Primary Tumor       RNA                   48          48
                               microRNA              47          47
TCGA-ESCA  Primary Tumor       RNA                  184         184
                               microRNA             184         186
           Solid Tissue Normal RNA                   13          13
                               microRNA              13          13
TCGA-GBM   Primary Tumor       RNA                  284         372
                               microRNA             267         272
           Solid Tissue Normal RNA                    5           5
                               microRNA               5           5
TCGA-HNSC  Primary Tumor       RNA                  520         520
                               microRNA             523         523
           Solid Tissue Normal RNA                   44          44
                               microRNA              44          44
TCGA-KICH  Primary Tumor       RNA                   66          66
                               microRNA              66          66
           Solid Tissue Normal RNA                   25          25
                               microRNA              25          25
TCGA-KIRC  Primary Tumor       RNA                  533         541
                               microRNA             516         544
           Solid Tissue Normal RNA                   72          72
                               microRNA              71          71
TCGA-KIRP  Primary Tumor       RNA                  290         290
                               microRNA             291         291
           Solid Tissue Normal RNA                   32          32
                               microRNA              34          34
TCGA-LGG   Primary Tumor       RNA                  516         516
                               microRNA             512         512
TCGA-LIHC  Primary Tumor       RNA                  371         371
                               microRNA             372         372
           Solid Tissue Normal RNA                   50          50
                               microRNA              50          50
TCGA-LUAD  Primary Tumor       RNA                  516         539
                               microRNA             513         519
           Solid Tissue Normal RNA                   59          59
                               mi

# Cases and Files

## Sample Types and Data File Types per Case

In [11]:
# Copy some columns of the EDA DataFrame
df_cases_eda = df_eda[['case_id', 'file_id', 'data_and_sample']].copy()

# Count the total number of files type
df_cases_eda['NTmRNA'] = np.where(
    df_cases_eda['data_and_sample'] == 'RNA-Normal', 1, 0
)
df_cases_eda['NTmicroRNA'] = np.where(
    df_cases_eda['data_and_sample'] == 'microRNA-Normal', 1, 0
)
df_cases_eda['TTmRNA'] = np.where(
    df_cases_eda['data_and_sample'] == 'RNA-Tumor', 1, 0
)
df_cases_eda['TTmicroRNA'] = np.where(
    df_cases_eda['data_and_sample'] == 'microRNA-Tumor', 1, 0
)

# Sum all count columns
df_cases_eda['file_count'] = (
    df_cases_eda['NTmRNA'] + df_cases_eda['NTmicroRNA'] + 
    df_cases_eda['TTmRNA'] + df_cases_eda['TTmicroRNA']
)

# Count the total number of files type per case
df_cases_eda = df_cases_eda \
    .groupby('case_id') \
    .agg(
        TTmRNA_count=pd.NamedAgg(column='TTmRNA', aggfunc='sum'),
        TTmicroRNA_count=pd.NamedAgg(column='TTmicroRNA', aggfunc='sum'),
        NTmRNA_count=pd.NamedAgg(column='NTmRNA', aggfunc='sum'),
        NTmicroRNA_count=pd.NamedAgg(column='NTmicroRNA', aggfunc='sum'),
        file_count=pd.NamedAgg(column='file_count', aggfunc='sum')
    ) \
    .sort_values(
        by=['file_count', 'TTmRNA_count', 'TTmicroRNA_count', 'NTmRNA_count', 'NTmicroRNA_count'],
        ascending=False
    )

In [12]:
# Print the total number of files type per case
df_cases_eda

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count
case_id,,,,,
556fcbc8-172a-4af1-8822-ae036e8d68e8,3,3,1,1,8
8c7e74e0-71ef-49b8-9217-94b8ef740ef9,3,3,1,1,8
f130f376-5801-40f9-975d-a7e2f7b5670d,3,3,1,1,8
0c0b610e-fe4c-406d-a5ed-5cc3b11dabf5,3,3,1,0,7
889aec8e-14ba-48d9-8fe1-f2416e82b333,3,3,1,0,7
...,...,...,...,...,...
f9824a6e-7a97-445c-8846-df8d8cddedaa,0,1,0,0,1
f9cf605f-f287-4eba-8e8f-8ba47e14ecc4,0,1,0,0,1
fdd4adb8-9295-480a-9352-305b5eb51187,0,1,0,0,1


## Case Types Summarization

In [13]:
# Calculate the normalized count of case types
count_abs = df_cases_eda.value_counts(ascending=False)
count_norm = df_cases_eda.value_counts(normalize=True, ascending=False)

# Summarizes the different types of cases
df_case_counts = pd.DataFrame({
    'count': count_abs,
    'count%': round(count_norm * 100, 2)
})

In [14]:
# Print the summarization of the different types of cases
df_case_counts.reset_index()

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count,count,count%
0,1,1,0,0,2,8867,86.87
1,1,1,1,1,4,594,5.82
2,1,0,0,0,1,177,1.73
3,0,1,0,0,1,168,1.65
4,1,1,1,0,3,98,0.96
5,2,1,0,0,3,88,0.86
6,0,2,0,0,2,48,0.47
7,1,1,0,1,3,38,0.37
8,1,2,0,0,3,35,0.34
9,0,0,1,1,2,17,0.17


# BRCA Project

## Data and Informations

In [15]:
# Query the data related to TCGA-BRCA 
df_brca_eda = df_eda \
    .query('project_id == "TCGA-BRCA"') \
    .merge(
        right=df_cases_eda,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Print TCGA-BRCA basic informations
pd.set_option('display.max_colwidth', 300)
df_projects.query('project_id == "TCGA-BRCA"')

,project_id,project_primary_site,project_disease_type,project_name,project_case_count
32,TCGA-BRCA,['Breast'],"['Epithelial Neoplasms, NOS', 'Adnexal and Skin Appendage Neoplasms', 'Squamous Cell Neoplasms', 'Adenomas and Adenocarcinomas', 'Complex Epithelial Neoplasms', 'Fibroepithelial Neoplasms', 'Cystic, Mucinous and Serous Neoplasms', 'Basal Cell Neoplasms', 'Ductal and Lobular Neoplasms']",Breast Invasive Carcinoma,1098


## Cases per Disease Type

In [16]:
# Count the number of cases per disease type
df_brca_eda \
    .groupby('case_disease_type') \
    .agg(
        count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
    ) \
    .sort_values(by='count', ascending=False)

,count
case_disease_type,
Ductal and Lobular Neoplasms,1053
"Cystic, Mucinous and Serous Neoplasms",16
Complex Epithelial Neoplasms,14
"Epithelial Neoplasms, NOS",5
Adenomas and Adenocarcinomas,3
Fibroepithelial Neoplasms,2
Squamous Cell Neoplasms,2
Adnexal and Skin Appendage Neoplasms,1
Basal Cell Neoplasms,1


## Case Types Summarization

In [17]:
# Drop the unnecessary columns and duplicated rows
columns = [
    'TTmRNA_count',
    'TTmicroRNA_count',
    'NTmRNA_count',
    'NTmicroRNA_count',
    'file_count'
]
df_brca_cases_eda = df_brca_eda.drop_duplicates(subset='case_id')[columns]

# Calculate the normalized count of case types
count_abs = df_brca_cases_eda.value_counts(ascending=False)
count_norm = df_brca_cases_eda.value_counts(normalize=True, ascending=False)

# Summarizes the different types of cases
pd.DataFrame({
    'count': count_abs,
    'count%': round(count_norm * 100, 2)
}).reset_index()

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count,count,count%
0,1,1,0,0,2,955,87.06
1,1,1,1,1,4,99,9.02
2,1,0,0,0,1,18,1.64
3,1,1,1,0,3,9,0.82
4,2,2,0,0,4,4,0.36
5,3,3,0,0,6,3,0.27
6,0,1,0,0,1,2,0.18
7,1,2,0,0,3,2,0.18
8,2,2,1,1,6,2,0.18
9,3,3,1,1,8,2,0.18


# CESC Project

## Data and Informations

In [18]:
# Query the data related to TCGA-CESC 
df_cesc_eda = df_eda \
    .query('project_id == "TCGA-CESC"') \
    .merge(
        right=df_cases_eda,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Print TCGA-CESC basic informations
pd.set_option('display.max_colwidth', 300)
df_projects.query('project_id == "TCGA-CESC"')

,project_id,project_primary_site,project_disease_type,project_name,project_case_count
16,TCGA-CESC,"['Ovary', 'Cervix uteri']","['Complex Epithelial Neoplasms', 'Adenomas and Adenocarcinomas', 'Squamous Cell Neoplasms', 'Cystic, Mucinous and Serous Neoplasms']",Cervical Squamous Cell Carcinoma and Endocervical Adenocarcinoma,307


## Cases per Disease Type

In [19]:
# Count the number of cases per disease type
df_cesc_eda \
    .groupby('case_disease_type') \
    .agg(
        count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
    ) \
    .sort_values(by='count', ascending=False)

,count
case_disease_type,
Squamous Cell Neoplasms,254
Adenomas and Adenocarcinomas,32
"Cystic, Mucinous and Serous Neoplasms",17
Complex Epithelial Neoplasms,4


## Case Types Summarization

In [20]:
# Drop the unnecessary columns and duplicated rows
columns = [
    'TTmRNA_count',
    'TTmicroRNA_count',
    'NTmRNA_count',
    'NTmicroRNA_count',
    'file_count'
]
df_cesc_cases_eda = df_cesc_eda.drop_duplicates(subset='case_id')[columns]

# Calculate the normalized count of case types
count_abs = df_cesc_cases_eda.value_counts(ascending=False)
count_norm = df_cesc_cases_eda.value_counts(normalize=True, ascending=False)

# Summarizes the different types of cases
pd.DataFrame({
    'count': count_abs,
    'count%': round(count_norm * 100, 2)
}).reset_index()

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count,count,count%
0,1,1,0,0,2,301,98.05
1,0,1,0,0,1,3,0.98
2,1,1,1,1,4,3,0.98


# OV Project

## Data and Informations

In [21]:
# Query the data related to TCGA-OV
df_ov_eda = df_eda \
    .query('project_id == "TCGA-OV"') \
    .merge(
        right=df_cases_eda,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Print TCGA-OV basic informations
pd.set_option('display.max_colwidth', 300)
df_projects.query('project_id == "TCGA-OV"')

,project_id,project_primary_site,project_disease_type,project_name,project_case_count
30,TCGA-OV,"['Ovary', 'Retroperitoneum and peritoneum']","['Cystic, Mucinous and Serous Neoplasms', 'Not Reported']",Ovarian Serous Cystadenocarcinoma,608


## Cases per Disease Type

In [22]:
# Count the number of cases per disease type
df_ov_eda \
    .groupby('case_disease_type') \
    .agg(
        count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
    ) \
    .sort_values(by='count', ascending=False)

,count
case_disease_type,
"Cystic, Mucinous and Serous Neoplasms",522


## Case Types Summarization

In [23]:
# Drop the unnecessary columns and duplicated rows
columns = [
    'TTmRNA_count',
    'TTmicroRNA_count',
    'NTmRNA_count',
    'NTmicroRNA_count',
    'file_count'
]
df_ov_cases_eda = df_ov_eda.drop_duplicates(subset='case_id')[columns]

# Calculate the normalized count of case types
count_abs = df_ov_cases_eda.value_counts(ascending=False)
count_norm = df_ov_cases_eda.value_counts(normalize=True, ascending=False)

# Summarizes the different types of cases
pd.DataFrame({
    'count': count_abs,
    'count%': round(count_norm * 100, 2)
}).reset_index()

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count,count,count%
0,1,1,0,0,2,420,80.46
1,0,1,0,0,1,58,11.11
2,0,2,0,0,2,43,8.24
3,1,2,0,0,3,1,0.19


# UCEC Project

## Data and Informations

In [24]:
# Query the data related to TCGA-UCEC
df_ucec_eda = df_eda \
    .query('project_id == "TCGA-UCEC"') \
    .merge(
        right=df_cases_eda,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Print TCGA-UCEC basic informations
pd.set_option('display.max_colwidth', 300)
df_projects.query('project_id == "TCGA-UCEC"')

,project_id,project_primary_site,project_disease_type,project_name,project_case_count
28,TCGA-UCEC,"['Corpus uteri', 'Uterus, NOS']","['Epithelial Neoplasms, NOS', 'Adenomas and Adenocarcinomas', 'Cystic, Mucinous and Serous Neoplasms', 'Not Reported']",Uterine Corpus Endometrial Carcinoma,560


## Cases per Disease Type

In [25]:
# Count the number of cases per disease type
df_ucec_eda \
    .groupby('case_disease_type') \
    .agg(
        count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
    ) \
    .sort_values(by='count', ascending=False)

,count
case_disease_type,
Adenomas and Adenocarcinomas,414
"Cystic, Mucinous and Serous Neoplasms",131
Not Reported,12
"Epithelial Neoplasms, NOS",2


## Case Types Summarization

In [26]:
# Drop the unnecessary columns and duplicated rows
columns = [
    'TTmRNA_count',
    'TTmicroRNA_count',
    'NTmRNA_count',
    'NTmicroRNA_count',
    'file_count'
]
df_ucec_cases_eda = df_ucec_eda.drop_duplicates(subset='case_id')[columns]

# Calculate the normalized count of case types
count_abs = df_ucec_cases_eda.value_counts(ascending=False)
count_norm = df_ucec_cases_eda.value_counts(normalize=True, ascending=False)

# Summarizes the different types of cases
pd.DataFrame({
    'count': count_abs,
    'count%': round(count_norm * 100, 2)
}).reset_index()

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count,count,count%
0,1,1,0,0,2,510,91.23
1,1,1,1,1,4,21,3.76
2,0,0,1,1,2,11,1.97
3,1,0,0,0,1,8,1.43
4,3,3,0,0,6,3,0.54
5,0,1,0,0,1,2,0.36
6,0,0,1,0,1,1,0.18
7,1,0,1,1,3,1,0.18
8,1,1,1,0,3,1,0.18
9,3,2,0,0,5,1,0.18


# UCS Project

## Data and Informations

In [27]:
# Query the data related to TCGA-UCS
df_ucs_eda = df_eda \
    .query('project_id == "TCGA-UCS"') \
    .merge(
        right=df_cases_eda,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Print TCGA-UCS basic informations
pd.set_option('display.max_colwidth', 300)
df_projects.query('project_id == "TCGA-UCS"')

,project_id,project_primary_site,project_disease_type,project_name,project_case_count
1,TCGA-UCS,"['Corpus uteri', 'Uterus, NOS']","['Complex Mixed and Stromal Neoplasms', 'Basal Cell Neoplasms']",Uterine Carcinosarcoma,57


## Cases per Disease Type

In [28]:
# Count the number of cases per disease type
df_ucs_eda \
    .groupby('case_disease_type') \
    .agg(
        count=pd.NamedAgg(column='case_id', aggfunc='nunique'),
    ) \
    .sort_values(by='count', ascending=False)

,count
case_disease_type,
Complex Mixed and Stromal Neoplasms,56
Basal Cell Neoplasms,1


## Case Types Summarization

In [29]:
# Drop the unnecessary columns and duplicated rows
columns = [
    'TTmRNA_count',
    'TTmicroRNA_count',
    'NTmRNA_count',
    'NTmicroRNA_count',
    'file_count'
]
df_ucs_cases_eda = df_ucs_eda.drop_duplicates(subset='case_id')[columns]

# Calculate the normalized count of case types
count_abs = df_ucs_cases_eda.value_counts(ascending=False)
count_norm = df_ucs_cases_eda.value_counts(normalize=True, ascending=False)

# Summarizes the different types of cases
pd.DataFrame({
    'count': count_abs,
    'count%': round(count_norm * 100, 2)
}).reset_index()

,TTmRNA_count,TTmicroRNA_count,NTmRNA_count,NTmicroRNA_count,file_count,count,count%
0,1,1,0,0,2,57,100.0
